<a href="https://colab.research.google.com/github/PanoEvJ/summarization_RLHF/blob/main/rlhf_PPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -q -U git+https://github.com/lvwerra/trl.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U git+https://github.com/huggingface/peft.git

!pip install -q transformers==4.30
!pip install -q -U sentencepiece
!pip install -q huggingface_hub
!pip install -q tdqm torch>=0.3.0
!pip install -q -U bitsandbytes
!pip install -q -U wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 95.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Insta

In [33]:
import os
import random
import wandb

from dataclasses import dataclass, field
from typing import Optional

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset as TorchDataset
from datasets import load_dataset, Dataset as HFDataset
from peft import (AutoPeftModelForCausalLM,
                  LoraConfig,
                  PeftConfig,
                  PeftModel,
                  TaskType,
                  get_peft_config,
                  get_peft_model,
                  get_peft_model_state_dict,
                  set_peft_model_state_dict,
                  PeftType,
)
from tqdm import tqdm
from transformers import (AutoModelForSeq2SeqLM,
                          AutoModelForSequenceClassification,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          TrainingArguments,
                          AutoTokenizer,
                          T5ForConditionalGeneration,
                          T5Tokenizer
)
from trl import (SFTTrainer,
                 PPOConfig,
                 PPOTrainer,
                 AutoModelForSeq2SeqLMWithValueHead,
                 create_reference_model,
                 set_seed
)
from trl.core import LengthSampler

## Set Cuda

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()

## Load the Rewards Model from Google Drive

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
rewards_model_directory = "./drive/MyDrive/rewards_model/"

rm_model = AutoModelForSequenceClassification.from_pretrained(rewards_model_directory)
rm_tokenizer = AutoTokenizer.from_pretrained(rewards_model_directory)
rm_model.to(device)

In [18]:
import torch.nn.functional as F


def score_summaries(model, tokenizer, chosen_summary, rejected_summary):
    # Tokenize the inputs
    chosen_tokens = tokenizer(chosen_summary, return_tensors="pt", padding='max_length', truncation=True, max_length=512)
    rejected_tokens = tokenizer(rejected_summary, return_tensors="pt", padding='max_length', truncation=True, max_length=512)

    chosen_tokens.to(device)
    rejected_tokens.to(device)

    # Get logits from the model
    with torch.no_grad():
        chosen_logits = model(**chosen_tokens).logits
        rejected_logits = model(**rejected_tokens).logits

    # Apply softmax to get probabilities
    chosen_probs = F.softmax(chosen_logits, dim=-1)
    rejected_probs = F.softmax(rejected_logits, dim=-1)

    # Assuming the positive class (indicating 'chosen' is good) is the second one
    chosen_score = chosen_probs[0][1].item()
    rejected_score = rejected_probs[0][1].item()

    # Extract logits for each summary
    chosen_logit = chosen_logits[0][1].item()
    rejected_logit = rejected_logits[0][1].item()

    return chosen_score, rejected_score, chosen_logit, rejected_logit

In [19]:
chosen_summary = "Water meter in another condo is not in our condo. What can we do legally to restore water to my condo complex?"
rejected_summary = "Go fix the problem."

In [20]:
chosen_score, rejected_score, chosen_logit, rejected_logit = score_summaries(rm_model, rm_tokenizer, chosen_summary, rejected_summary)

print(f"Chosen Score: {chosen_score:.4f}")
print(f"Rejected Score: {rejected_score:.4f}")

print(f"Chosen Logit: {chosen_logit:.4f}")
print(f"Rejected Logit: {rejected_logit:.4f}")

Chosen Score: 0.4960
Rejected Score: 0.5220
Chosen Logit: -0.0216
Rejected Logit: 0.0866


## Load the T5 model

In [22]:
policy_model_name = "t5-base"
policy_model_id = 'PanoEvJ/T5_base_SFT_summarization'

In [ ]:
# bnb_config = BitsAndBytesConfig(
#     load_in_8bit=True,
#     bnb_8bit_use_double_quant=True,
#     bnb_8bit_quant_type="nf8",
#     bnb_8bit_compute_dtype=torch.bfloat16,
# )

In [23]:
policy_model = T5ForConditionalGeneration.from_pretrained(policy_model_id)
policy_model.to(device)
policy_tokenizer = T5Tokenizer.from_pretrained(policy_model_id)

In [24]:
task_prefix = "summarize: "

text = "SUBREDDIT: r/relationships TITLE: How do I/do I at all [20 F] tell my boyfriend [23 M] that I'm bisexual? POST: I've had two serious relationships prior to this one, both with women. They had no problem with me being bisexual and it was something known before the relationship -- my first girlfriend was also bisexual. I am now in a relationship with a guy. We've been exclusive for about a month. Having never faced this issue, I come to you, Reddit. Is this something that he needs to know? Is it really relevant to a hetero relationship, regardless of if one of the participants in the relationship is bisexual? If you guys think it is necessary, when do you think is the right time? I think my biggest fear is losing him because of it. I know that I should be with someone who is fine with who I am, but I really like the guy and I'd hate for my sexual orientation to be the thing that kills this."
#text = "SUBREDDIT: r/legaladvice TITLE: What can I do legally to restore water to my condominium!? POST: Hi, I live in SE Michigan in a condominium complex. Our water was shut off due to non-payment. (we recieved no notice) and we had to pay all that was due ($1500) We payed this yesterday at 2, they said the water would be turned on immediately. It wasn't. It's now the next day. The lady in our assosciation keeps insisting that the water meter is in another condo. Which we can't access because the person living there is never there (it's being rented) Now we're stuck with no water, no shower, no teeth brushing, no toilets, and no food for certain meals.... Please help us... What can we do? We called the police and they say that we can file a civil report for the lady not doing her job..."
prompt = f"{task_prefix}{text}"
input_ids = policy_tokenizer(prompt, return_tensors="pt").input_ids.to(device)
outputs = policy_model.generate(input_ids, max_length=100).to(device)

strOutput = policy_tokenizer.decode(outputs[0], skip_special_tokens=True)
print(strOutput)

chosen_score, rejected_score, chosen_logit, rejected_logit = score_summaries(rm_model, rm_tokenizer, strOutput, "")

print(f"Chosen Score: {chosen_score:.4f}")
print(f"Rejected Score: {rejected_score:.4f}")

print(f"Chosen Logit: {chosen_logit:.4f}")
print(f"Rejected Logit: {rejected_logit:.4f}")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1259: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


SUBREDDIT: r/relationships TITLE: How do I/do I at all [20 F] tell my boyfriend [23 M] that I'm bisexual? POST: I've had two serious relationships prior to this one, both with women. They had no problem with me being bisexual and it was something known before the relationship -- my first girlfriend was also bisexual. I am now in a relationship with a
Chosen Score: 0.5220
Rejected Score: 0.4793
Chosen Logit: 0.0167
Rejected Logit: -0.0410


### Load the Peft Adapters

In [ ]:
# see the available modules by printint out the model
print(policy_model)

In [25]:
lora_config = LoraConfig(
    r=8, # Rank
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.10,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM # T5
)

In [ ]:
policy_peft_model = get_peft_model(policy_model, lora_config)
policy_peft_model.to(device)

In [46]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

print_trainable_parameters(policy_model)

trainable params: 884736 || all params: 223788288 || trainable%: 0.3953450861557152


### Load the Value Head used for the PPOTrainer

In [ ]:
ppo_model = AutoModelForSeq2SeqLMWithValueHead.from_pretrained(policy_peft_model,
                                                               torch_dtype=torch.bfloat16,
                                                               is_trainable=True)

ppo_model.to(device)

### Create Reference Model

In [ ]:
ref_model = create_reference_model(policy_model)
ref_model.to(device)

## Load and Prepare the Dataset

In [37]:
# Load the dataset
orig_dataset = load_dataset('CarperAI/openai_summarize_comparisons', split='train')

# Filter samples where the prompt length is less than or equal to 750
filtered_dataset = orig_dataset.filter(lambda example: len(example['prompt'].split()) <= 450) # By word
#filtered_dataset = orig_dataset.filter(lambda example: len(example['prompt']) <= 1250) # By character

# Shuffle and select the first 10K samples
#shuffled_dataset = orig_dataset.shuffle(seed=42).select(range(1000))
shuffled_dataset = filtered_dataset.shuffle(seed=42).select(range(2000))


# Extract the desired features.  Renaming chose to response to follow the ppo library requirements.
new_dataset_dict = {
    "prompt": shuffled_dataset["prompt"],
    "response": shuffled_dataset["chosen"]
}

# Convert the dictionary to a new Dataset
dataset = HFDataset.from_dict(new_dataset_dict)

# Split the new_dataset into train_dataset and eval_dataset
# split_ratio = 0.8  # 80% for training, 20% for evaluation
dataset_split = dataset.train_test_split(test_size=0.8)
train_dataset = dataset_split['train']
eval_dataset = dataset_split['test']

In [38]:
print(train_dataset[0].keys())
print(eval_dataset[0].keys())

dict_keys(['prompt', 'response'])
dict_keys(['prompt', 'response'])


Tokenize the dataset

In [39]:
# Instantiate your tokenizer (replace T5Tokenizer with your model's tokenizer if different)
tokenizer = T5Tokenizer.from_pretrained("t5-small") # or whatever model you're using

def tokenize_function(example):
    # Tokenize the prompt and store it as input_ids. Also return the response.
    return {
        "input_ids": tokenizer(example["prompt"], return_tensors="pt", truncation=True, max_length=512)["input_ids"].squeeze(),
        "response": example["response"],
    }

# Tokenize the training and evaluation datasets
train_dataset = train_dataset.map(tokenize_function, batched=False)
eval_dataset = eval_dataset.map(tokenize_function, batched=False)

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

In [40]:
train_dataset

Dataset({
    features: ['prompt', 'response', 'input_ids'],
    num_rows: 400
})

In [41]:
print(train_dataset[0]) # check the input_ids

{'prompt': "SUBREDDIT: r/relationships\nTITLE: Me [28 M] with my ex-GF [26 F] 5 years, broke up 6 months ago and still missing her\nPOST: She broke up with me 6 months ago and I still miss her. It was my birthday a few days ago and she didn't wish me a happy birthday. I know that she remembered because it is a few days before her birthday. I didn't bother sending her a happy birthday either.\n\nI really missed her today. I went through out photos and had a bit of a cry. \n\nI went out with this girl at work. She seemed interested in hanging out with me. When we were driving around, I thought a lot about my ex. This new girl was touching my arm and getting close to me. I'm not sure if she likes me - it still feels so foreign to me. I'm not ready to get into a new relationship, but I'd like to move on.", 'response': "TL;DR:  Ex-GF that I still miss 6 months later, I went out with a girl at work and I'm not sure if she likes me.", 'input_ids': [3, 4138, 25582, 11253, 3177, 10, 3, 52, 87, 

## PPO Trainer

In [42]:
def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

test_data = [{"key1": "value1", "key2": "value2", "key3": "value3"}, {"key1": "value4", "key2": "value5", "key3": "value6"}]
print(f'Collator input: {test_data}')
print(f'Collator output: {collator(test_data)}')

Collator input: [{'key1': 'value1', 'key2': 'value2', 'key3': 'value3'}, {'key1': 'value4', 'key2': 'value5', 'key3': 'value6'}]
Collator output: {'key1': ['value1', 'value4'], 'key2': ['value2', 'value5'], 'key3': ['value3', 'value6']}


In [43]:
# Lets sample what the collator generates:
sample_data = [train_dataset[i] for i in range(3)]  # take first three examples
collated_data = collator(sample_data)
print(collated_data.keys())

dict_keys(['prompt', 'response', 'input_ids'])


In [45]:
ppo_config = PPOConfig(
    steps=512,
    model_name=policy_model,
    learning_rate=1e-4,
    batch_size=128,
    mini_batch_size=16,
    gradient_accumulation_steps=1,
    optimize_cuda_cache=True,
    ppo_epochs=8,
    target_kl=0.1
)

In [48]:
ppo_trainer = PPOTrainer(config=ppo_config,
                         model=ppo_model,
                         ref_model=ref_model,
                         tokenizer=policy_tokenizer,
                         dataset=train_dataset,
                         data_collator=collator)

## Fine-tune the model with PPO

In [ ]:
for query_tensor, response_tensor in dummy_dataloader:
    # define a reward for response
    # (this could be any reward such as human feedback or output from another model)
    reward = [torch.tensor(1.0), torch.tensor(0.0)]
    # train model
    train_stats = ppo_trainer.step([q for q in query_tensor], [r for r in response_tensor], reward)
    break

In [ ]:
    question_tensors = batch["input_ids"]

    response_tensors = ppo_trainer.generate(
        question_tensors,
        return_prompt=False,
        length_sampler=output_length_sampler,
        **generation_kwargs,
    )
    batch["response"] = tokenizer.batch_decode(response_tensors, skip_special_tokens=True)

    # Compute reward score (using the sentiment analysis pipeline)
    texts = [q + r for q, r in zip(batch["query"], batch["response"])]
    pipe_outputs = sentiment_pipe(texts, **sent_kwargs)
    rewards = [torch.tensor(output[0]["score"] - script_args.reward_baseline) for output in pipe_outputs]

In [ ]:
for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    if epoch >= config.total_ppo_epochs:
        break


In [ ]:
# Run PPO step
stats = ppo_trainer.step(question_tensors, response_tensors, rewards)

In [ ]:
# Log stats to Wandb
ppo_trainer.log_stats(stats, batch, rewards)

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("./rewards_model/")
tokenizer = AutoTokenizer.from_pretrained("./rewards_model/")

NameError: ignored